# Load Tasks and VASP Files From NERSC

### Load Imports And Open Maggma Stores

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%run imports.py

/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/ext/matproj.py:454: DeprecationWarning:

__init__ is deprecated
MaterialsProjectCompatibility will be updated with new correction classes as well as new values of corrections and uncertainties in 2020



Imports successfully loaded


In [4]:
working_dir = os.getcwd()

In [5]:
%run maggma_stores.py

All maggma stores (db, elfcar_store, chgcar_store, aeccar0_store, aeccar2_store) successfully loaded 
Run 'connect_to_stores()' to connect to stores


In [6]:
connect_to_stores()

All connections successful


### Retrieve Tasks From NERSC

In [7]:
tasks_iter = db.query({'tags':{'$regex':'production-scan'}})
tasks = [t for t in tasks_iter]
len(tasks)

8295

### Sort Element and Binary Compound Tasks

In [8]:
is_binary = lambda f: len(Composition(f).elements) == 2
binary_tasks = [t for t in tasks if is_binary(t['formula_pretty'])]
len(binary_tasks)

5726

In [9]:
tasks_dict = {}

for t in binary_tasks:
    f = t['formula_pretty']
    if f not in tasks_dict.keys():
        tasks_dict[f] = t
    elif t['output']['energy_per_atom'] < tasks_dict[f]['output']['energy_per_atom']:
        tasks_dict[f] = t

In [10]:
is_element = lambda f: len(Composition(f).elements) == 1
element_tasks = [t for t in tasks if is_element(t['formula_pretty'])]
len(element_tasks)

712

In [11]:
elem_tasks_dict = {}

for t in element_tasks:
    f = re.sub(r'\d+', '', t['formula_pretty'])
    if f not in elem_tasks_dict.keys():
        elem_tasks_dict[f] = t
    elif t['output']['energy_per_atom'] < elem_tasks_dict[f]['output']['energy_per_atom']:
        elem_tasks_dict[f] = t

In [15]:
elem_data_json = {f:elem_tasks_dict[f] for f in elem_tasks_dict.keys()}
elem_ids = {f:elem_data_json[f]['task_id'] for f in elem_data_json.keys()}

### Find Subset of Binary Compounds to Study 
#### Formula must: (1) be in SCAN calculations, (2) be in JANAF experimental data, (3) match specified anion/cation criteria 

In [13]:
scan_formulas = set([Composition(t['formula_pretty']) for t in binary_tasks])
len(scan_formulas)

4943

In [16]:
janaf_formulas = []

for f in pd.read_csv('data/janaf_thermo.csv')['Formula'].tolist():
    try:
        janaf_formulas.append(Composition(f))
    except:
        print(f)
        
janaf_formulas = set(janaf_formulas)

e-


In [17]:
len(scan_formulas & janaf_formulas)

178

In [18]:
task_ids = [t['task_id'] for t in tasks_dict.values()]

In [19]:
anions = set([Element(e) for e in ['N', 'O', 'F', 'Cl', 'Br']])
cats = ['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Ni'] + ['Li', 'Na', 'K', 'Rb', 'Cs'] + ['Be', 'Mg', 'Ca', 'Sr', 'Ba']
cations = set([Element(e) for e in cats])

In [20]:
selection = [f for f in (scan_formulas & janaf_formulas) if \
             set(Composition(f).elements) & anions or set(Composition(f).elements) & cations]
len(selection)

160

In [21]:
selected_formulas = [c.reduced_formula for c in selection]
print(selected_formulas)

['LiH', 'Be2C', 'Be3N2', 'BN', 'BeO', 'B2O3', 'BeF2', 'Li3N', 'MgB2', 'MgB4', 'AlN', 'Al2O3', 'AlF3', 'BeS', 'LiCl', 'BeCl2', 'TiB2', 'KF', 'TiB', 'AlCl3', 'TiH2', 'KO2', 'TiC', 'CaO', 'NaCl', 'CaF2', 'MgCl2', 'Cr3C2', 'BeBr2', 'Cr2O3', 'FeO', 'Fe2O3', 'CoO', 'NaBr', 'MgBr2', 'AlBr3', 'FeS', 'FeS2', 'FeCl2', 'FeCl3', 'CoCl2', 'KBr', 'CaBr2', 'SrF2', 'BeI2', 'TiBr3', 'TiBr4', 'SrS', 'FeBr2', 'SrCl2', 'Li2O', 'BaO', 'BaF2', 'NbCl5', 'MoCl5', 'LiF', 'NaH', 'CsF', 'BaS', 'BaCl2', 'SrBr2', 'ZrBr3', 'MoBr2', 'MoBr3', 'ICl', 'CsCl', 'KI', 'CaI2', 'ZrBr4', 'TiI3', 'NbBr5', 'FeI2', 'Ta2O5', 'WO2', 'WO3', 'BaBr2', 'HgO', 'HgF2', 'TaCl5', 'PbO2', 'Pb3O4', 'HgCl2', 'WCl2', 'PbCl2', 'WCl4', 'Mg3N2', 'NaO2', 'MgF2', 'KH', 'BaI2', 'WBr6', 'P3N5', 'SiO2', 'Si3N4', 'HgBr2', 'PbBr2', 'Mg2Si', 'K2O', 'K2O2', 'Na2S', 'MgS', 'TiN', 'Cr7C3', 'Cr23C6', 'VN', 'TiO', 'TiO2', 'CrN', 'Cr2N', 'TiF3', 'TiF4', 'Ti2O3', 'V2O3', 'Fe3O4', 'Co3O4', 'FeF2', 'FeF3', 'K2S', 'CaS', 'KCl', 'CoF3', 'CaCl2', 'CuO', 'Cu2O', 'L

In [22]:
selected_ids = [tasks_dict[f]['task_id'] for f in selected_formulas]
print(selected_ids)

[6405, 3890, 7373, 11133, 3914, 5804, 10090, 3957, 3996, 6135, 3955, 4648, 4827, 3901, 2779, 6017, 4392, 3992, 4837, 6454, 4601, 4826, 3929, 3926, 3930, 4019, 6412, 5874, 6240, 5312, 2936, 10792, 4617, 4102, 4782, 8414, 5364, 4870, 7394, 10278, 10118, 4406, 6359, 4026, 10408, 9789, 9416, 2180, 4999, 4510, 3947, 4072, 2072, 9887, 10201, 2597, 3882, 4212, 4226, 4682, 9493, 7850, 9879, 8450, 8661, 4331, 4687, 5723, 10375, 8706, 10389, 5431, 7076, 7562, 10285, 7722, 5662, 4089, 9731, 4656, 9068, 8428, 9914, 6190, 7962, 10642, 5462, 4611, 4035, 7921, 7740, 6344, 5557, 5570, 6853, 6417, 4283, 4311, 5692, 4252, 3942, 3989, 9927, 8986, 4758, 2167, 4787, 2820, 7468, 8989, 10948, 6283, 7408, 11087, 6862, 4757, 6094, 2085, 4079, 4346, 7239, 5863, 6872, 4493, 4644, 5822, 10674, 9327, 7178, 4800, 7019, 6111, 5298, 6936, 3979, 4018, 6355, 8994, 8091, 8563, 10373, 10179, 9534, 4319, 5278, 7051, 8061, 7958, 10101, 4298, 5275, 4201, 8381, 9665, 4108, 10524, 4002, 5663, 3907, 3915]


In [23]:
binary_ids = {selected_formulas[i]:selected_ids[i] for i in range(len(selected_formulas))}

### Load ELFCARs, CHGCARs, and AECCARs (0 and 2)

In [25]:
# Binary Compounds

car_dict = {}

for ID in tqdm(selected_ids):
    elfcar = [e for e in elfcar_store.query({"metadata.task_id":ID})][0]
    if elfcar.get("data_aug"):
        del elfcar["data_aug"]
    ELF = Elfcar.from_dict(elfcar)
    
    chgcar = [e for e in chgcar_store.query({"metadata.task_id":ID})][0]
    CHG = Chgcar.from_dict(chgcar)

    aec0 = [e for e in aeccar0_store.query({"metadata.task_id":ID})][0]
    if aec0.get("data_aug"):
        del aec0["data_aug"]
    AEC0 = Chgcar.from_dict(aec0)

    aec2 = [e for e in aeccar2_store.query({"metadata.task_id":ID})][0]
    if aec2.get("data_aug"):
        del aec2["data_aug"] # bug fix line
    AEC2 = Chgcar.from_dict(aec2)
    
    car_dict[ID] = [ELF, CHG, AEC0, AEC2]


 97%|█████████▋| 155/160 [15:29<00:31,  6.33s/it]


100%|██████████| 160/160 [15:47<00:00,  5.92s/it]


In [26]:
elem_elfcars = {}

for ID in tqdm(elem_ids.values()):
    elfcar = [e for e in elfcar_store.query({"metadata.task_id":ID})][0]
    if elfcar.get("data_aug"):
        del elfcar["data_aug"]
    ELF = Elfcar.from_dict(elfcar)
    elem_elfcars[ID] = ELF


100%|██████████| 88/88 [00:21<00:00,  4.03it/s]


### Write Files Locally

In [27]:
# Binary Compounds

for i in tqdm(car_dict.keys()):
    label = str(i)
    lst = car_dict[i]
    filepath = 'files/'
    lst[0].write_file(filepath + 'ELFCAR_{0}'.format(str(i)))
    lst[1].write_file(filepath + 'CHGCAR_{0}'.format(str(i)))
    try:
        chgref = lst[2] + lst[3]
        chgref.write_file(filepath + 'CHGREF_{}'.format(str(i)))
    except ValueError:
        pass
    
dumpfn(binary_ids, 'data/binary_formulas_ids.json')


  0%|          | 0/160 [00:00<?, ?it/s]/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/io/vasp/outputs.py:3010: UserWarning:

Structures are different. Make sure you know what you are doing...


 95%|█████████▌| 152/160 [31:24<00:58,  7.37s/it]


100%|██████████| 160/160 [33:07<00:00, 12.42s/it]


In [33]:
# Elements

for i in tqdm(elem_ids.values()):
    elfcar = elem_elfcars[i]
    filepath = 'files/'
    elfcar.write_file(filepath + 'ELFCAR_{0}'.format(str(i)))
    
dumpfn(elem_ids, 'data/element_formulas_ids.json')


 20%|██        | 18/88 [00:04<00:22,  3.17it/s]/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/core/periodic_table.py:438: UserWarning:

No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.


 39%|███▊      | 34/88 [00:13<00:20,  2.58it/s]/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/core/periodic_table.py:438: UserWarning:

No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.


100%|██████████| 88/88 [00:42<00:00,  2.05it/s]
